# Testing Cellmage with Gemini 2.5 Flash

This notebook tests the capabilities of Cellmage with Google's Gemini 2.5 Flash model.

**Date:** April 24, 2025

## Setup and Configuration

Let's set up our development environment and import the necessary modules.

In [1]:
# Setup environment
import os
import sys
import logging

# Skip dotenv loading for testing
os.environ["CELLMAGE_SKIP_DOTENV"] = "1"

# Set up logging
logging.basicConfig(level=logging.INFO)

# Ensure the cellmage package can be imported
# Get the absolute path of the current working directory
notebook_dir = os.getcwd()
# Get the project root directory (parent of the notebook directory)
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))

print(f"Notebook directory: {notebook_dir}")
print(f"Project root directory: {project_root}")

if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added path: {project_root}")

try:
    # Import cellmage
    import cellmage

    # Check version - handle case where __version__ might not be available
    try:
        print(f"Cellmage version: {cellmage.__version__}")
    except AttributeError:
        print("Cellmage imported successfully, but version information is not available")
except ModuleNotFoundError as e:
    print(f"Error importing cellmage: {e}")
    print("\nDebug information:")
    print(f"Current working directory: {os.getcwd()}")
    print(f"Python path: {sys.path}")
    print("\nTry running this notebook from the project root directory")

2025-04-24 07:04:49,529 - cellmage - INFO - Cellmage logging initialized


Notebook directory: /Users/tpinto/madpin/cellmage/notebooks
Project root directory: /Users/tpinto/madpin/cellmage
Added path: /Users/tpinto/madpin/cellmage
Cellmage version: 0.1.0


## Setting Up the Gemini LLM Client

Now we'll configure the LLM client to use Google's Gemini 2.5 Flash model.

In [2]:
from cellmage.adapters.direct_client import DirectLLMAdapter

# Create an LLM client with Gemini-2.5-flash model
# Note: You need to set GOOGLE_API_KEY in your environment
llm_client = DirectLLMAdapter(default_model="gemini-2.5-flash")

print("LLM client initialized with model: gemini-2.5-flash")

2025-04-24 07:04:49,576 - cellmage.adapters.direct_client - INFO - [Override] Setting 'api_key' = sk-L...xxmA
2025-04-24 07:04:49,577 - cellmage.adapters.direct_client - INFO - [Override] Setting 'api_base' = https://litellm.oracle.madpin.dev
2025-04-24 07:04:49,577 - cellmage.adapters.direct_client - INFO - [Override] Setting 'model' = gemini-2.5-flash


LLM client initialized with model: gemini-2.5-flash


## Creating Specialized Personas for Gemini

Let's create some specialized personas that work well with Gemini models.

In [3]:
from cellmage.resources.memory_loader import MemoryLoader
from cellmage.storage.memory_store import MemoryStore

# Create in-memory components for testing
persona_loader = MemoryLoader()
snippet_provider = MemoryLoader()
history_store = MemoryStore()

# Create specialized personas
persona_loader.add_persona(
    name="code_expert",
    system_message="You are a programming expert who specializes in writing clean, efficient code. Provide detailed explanations and always suggest best practices.",
    config={"temperature": 0.3, "top_p": 0.95},
)

persona_loader.add_persona(
    name="creative_writer",
    system_message="You are a creative writing assistant who helps draft engaging stories and content. Be imaginative and suggest unique perspectives.",
    config={"temperature": 0.9, "top_p": 0.98},
)

# List available personas
print(f"Available personas: {persona_loader.list_personas()}")

2025-04-24 07:04:49,677 - cellmage.resources.memory_loader - INFO - Added persona 'code_expert' to memory
2025-04-24 07:04:49,678 - cellmage.resources.memory_loader - INFO - Added persona 'creative_writer' to memory


Available personas: ['code_expert', 'creative_writer']


## Creating a Chat Manager with Gemini

Now let's set up a Chat Manager with the Gemini model and our personas.

In [4]:
# Create a chat manager
chat_manager = cellmage.ChatManager(
    llm_client=llm_client,
    persona_loader=persona_loader,
    snippet_provider=snippet_provider,
    history_store=history_store,
)

# Set default persona to code_expert
chat_manager.set_default_persona("code_expert")

print("Chat manager initialized with 'code_expert' persona")

2025-04-24 07:04:49,688 - cellmage.chat_manager - INFO - Initializing ChatManager
2025-04-24 07:04:49,696 - cellmage.chat_manager - INFO - ChatManager initialized
2025-04-24 07:04:49,700 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.3
2025-04-24 07:04:49,701 - cellmage.adapters.direct_client - INFO - [Override] Setting 'top_p' = 0.95
2025-04-24 07:04:49,702 - cellmage.chat_manager - INFO - Default persona set to 'code_expert'


Chat manager initialized with 'code_expert' persona


## Testing Code Generation with Gemini

Let's test Gemini's code generation capabilities through our chat manager.

In [5]:
# Add a challenging code generation task
response = chat_manager.chat(
    """Write a Python function that acts as a simple text-based calculator. 
    It should take a string expression like '2 + 3 * 4' and return the calculated result.
    Handle addition, subtraction, multiplication, division, and parentheses.
    Make sure to respect the order of operations.""",
    stream=True,
)

2025-04-24 07:04:49,708 - cellmage.chat_manager - INFO - Sending message to LLM with 2 messages in context
2025-04-24 07:04:49,709 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 2 messages


## Testing Creative Content Generation

Now let's switch to our creative writer persona and test content generation.

In [6]:
# Switch to creative_writer persona
chat_manager.set_default_persona("creative_writer")
print("Switched to 'creative_writer' persona")

# Generate creative content
response = chat_manager.chat(
    """Write a short story about a programmer who discovers an AI that can 
    predict the future, but only for trivial events. Make it humorous and 
    include a surprising twist at the end.""",
    stream=True,
)

2025-04-24 07:05:26,885 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.9
2025-04-24 07:05:26,887 - cellmage.adapters.direct_client - INFO - [Override] Setting 'top_p' = 0.98
2025-04-24 07:05:26,887 - cellmage.chat_manager - INFO - Default persona set to 'creative_writer'
2025-04-24 07:05:26,888 - cellmage.chat_manager - INFO - Sending message to LLM with 4 messages in context
2025-04-24 07:05:26,888 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 4 messages


Switched to 'creative_writer' persona


## Adding Context with Code Snippets

Let's test how Gemini performs with code snippets as context.

In [7]:
# Switch back to code_expert persona
chat_manager.set_default_persona("code_expert")

# Create a code snippet with a bug
snippet_provider.add_snippet(
    name="buggy_sort",
    content="""```python
def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j], arr[j+1]  # Bug: This line is correct but there's a logical error
    return arr

# Test the function
test_array = [64, 34, 25, 12, 22, 11, 90]
sorted_array = bubble_sort(test_array)
print("Sorted array:", sorted_array)
```""",
)

# Add the snippet to the conversation
chat_manager.add_snippet("buggy_sort")

# Ask about the bug
response = chat_manager.chat(
    """There's a logical error in this bubble sort implementation. 
    The function seems to work correctly for this test case, but it's not implemented optimally. 
    Can you identify the issue and suggest a fix?""",
    stream=True,
)
response

2025-04-24 07:06:11,277 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.3
2025-04-24 07:06:11,279 - cellmage.adapters.direct_client - INFO - [Override] Setting 'top_p' = 0.95
2025-04-24 07:06:11,280 - cellmage.chat_manager - INFO - Default persona set to 'code_expert'
2025-04-24 07:06:11,281 - cellmage.resources.memory_loader - INFO - Added snippet 'buggy_sort' to memory
2025-04-24 07:06:11,281 - cellmage.chat_manager - INFO - Added snippet 'buggy_sort' as system message
2025-04-24 07:06:11,282 - cellmage.chat_manager - INFO - Sending message to LLM with 7 messages in context
2025-04-24 07:06:11,282 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 7 messages


''

## Comparing Gemini's Performance

Let's create a complex query and test it with temperature variations to see how Gemini's responses differ.

In [8]:
# Set a lower temperature for precise answers
import cellmage.integrations


chat_manager.set_override("temperature", 0.1)

# Complex technical question
tech_query = """Explain the differences between synchronous and asynchronous programming 
                 paradigms in Python, including when to use each and their advantages 
                 and disadvantages. Include code examples."""

# Get response with low temperature
print("===== Response with temperature = 0.1 =====")
precise_response = chat_manager.chat(tech_query, stream=False)

cellmage.integrations.ipython_magic.display(precise_response)
# Set a higher temperature for more creative answers
chat_manager.set_override("temperature", 0.9)

# Clear history to get a fresh response
chat_manager.clear_history()
chat_manager.set_default_persona("code_expert")

# Get response with high temperature
print("\n===== Response with temperature = 0.9 =====")
creative_response = chat_manager.chat(tech_query, stream=True)
# creative_response

2025-04-24 07:06:27,726 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.1
2025-04-24 07:06:27,727 - cellmage.chat_manager - INFO - Sending message to LLM with 9 messages in context
2025-04-24 07:06:27,728 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 9 messages


===== Response with temperature = 0.1 =====


'You are absolutely right! There is indeed a significant logical error in the provided bubble sort implementation, even though the test case might misleadingly appear to work (it actually just returns the original unsorted array because no swaps happen).\n\nLet\'s break down the issue and how to fix it.\n\n**The Logical Error**\n\nThe core of the bubble sort algorithm is the **swapping** of adjacent elements if they are in the wrong order. The provided code checks the condition `if arr[j] > arr[j+1]:`, which is correct for determining if a swap is needed. However, the line intended to perform the swap is:\n\n```python\narr[j], arr[j+1] = arr[j], arr[j+1]\n```\n\nThis line does **not** perform a swap. It simply assigns the value of `arr[j]` back to `arr[j]` and the value of `arr[j+1]` back to `arr[j+1]`. In essence, it does nothing to change the order of the elements in the array.\n\nBecause no swaps ever occur, the array remains in its original unsorted state after the function complet

2025-04-24 07:06:42,763 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.9
2025-04-24 07:06:42,765 - cellmage.history_manager - INFO - History cleared. Kept 2 system messages.
2025-04-24 07:06:42,766 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.3
2025-04-24 07:06:42,766 - cellmage.adapters.direct_client - INFO - [Override] Setting 'top_p' = 0.95
2025-04-24 07:06:42,766 - cellmage.chat_manager - INFO - Default persona set to 'code_expert'
2025-04-24 07:06:42,767 - cellmage.chat_manager - INFO - Sending message to LLM with 3 messages in context
2025-04-24 07:06:42,767 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 3 messages



===== Response with temperature = 0.9 =====


## Testing Gemini's Contextual Understanding

Let's test Gemini's ability to maintain context over a multi-turn conversation.

In [9]:
# Clear history for a fresh conversation
chat_manager.clear_history()
chat_manager.set_default_persona("code_expert")

# Let's have a multi-turn conversation about a specific topic
print("===== First Turn =====")
chat_manager.chat("What is a recursive function in programming?", stream=False)

print("\n===== Second Turn =====")
chat_manager.chat("Can you give me an example using Python?", stream=False)

print("\n===== Third Turn =====")
chat_manager.chat("What are some ways to optimize it to avoid stack overflow?", stream=False)

print("\n===== Fourth Turn =====")
chat_manager.chat("Implement one of those optimization techniques in the example you provided.", stream=False)

2025-04-24 07:07:06,608 - cellmage.history_manager - INFO - History cleared. Kept 2 system messages.
2025-04-24 07:07:06,609 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.3
2025-04-24 07:07:06,610 - cellmage.adapters.direct_client - INFO - [Override] Setting 'top_p' = 0.95
2025-04-24 07:07:06,610 - cellmage.chat_manager - INFO - Default persona set to 'code_expert'
2025-04-24 07:07:06,611 - cellmage.chat_manager - INFO - Sending message to LLM with 3 messages in context
2025-04-24 07:07:06,612 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 3 messages


===== First Turn =====


2025-04-24 07:07:19,293 - cellmage.chat_manager - INFO - Sending message to LLM with 5 messages in context
2025-04-24 07:07:19,294 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 5 messages



===== Second Turn =====


2025-04-24 07:07:28,900 - cellmage.chat_manager - INFO - Sending message to LLM with 7 messages in context
2025-04-24 07:07:28,902 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 7 messages



===== Third Turn =====


2025-04-24 07:07:55,327 - cellmage.chat_manager - INFO - Sending message to LLM with 9 messages in context
2025-04-24 07:07:55,328 - cellmage.adapters.direct_client - INFO - Calling model 'gemini-2.5-flash' with 9 messages



===== Fourth Turn =====


'Okay, let\'s implement the **Memoization** technique using Python\'s built-in `functools.lru_cache` decorator. This is the most common and idiomatic way to add memoization to a recursive function in Python.\n\nWe\'ll take the original recursive Fibonacci function and add the decorator.\n\n```python\nimport functools\n# import sys # Optional: Uncomment to potentially increase recursion depth for testing, use with caution!\n# sys.setrecursionlimit(2000)\n\n@functools.lru_cache(max_size=None) # This is the memoization decorator\ndef fibonacci_memoized(n):\n    """\n    Calculates the n-th Fibonacci number using recursion with memoization.\n    This version is significantly faster than the naive recursive version\n    for larger n because it avoids redundant calculations.\n    """\n    # 1. Base Cases:\n    # These are the stopping conditions.\n    if n <= 1:\n        return n\n\n    # 2. Recursive Step:\n    # The function calls itself with smaller values.\n    # lru_cache automatically 

## Saving Multi-Turn Conversations

Let's save our multi-turn conversation and see how it's stored.

In [10]:
# Save the conversation
save_path = chat_manager.save_conversation("gemini_recursion_conversation")
print(f"Conversation saved to: {save_path}")

# Clear the history
chat_manager.clear_history()
print(f"History cleared, current message count: {len(chat_manager.get_history())}")

# Load the conversation back
if save_path:
    chat_manager.load_conversation(save_path)
    loaded_history = chat_manager.get_history()
    print(f"Conversation loaded, message count: {len(loaded_history)}")

    # Display the loaded conversation summary
    print("\n===== Loaded Conversation Summary =====")
    for i, msg in enumerate(loaded_history):
        if msg.role != "system":
            content_preview = msg.content[:50] + "..." if len(msg.content) > 50 else msg.content
            print(f"[{msg.role}] {content_preview}")

2025-04-24 07:08:05,408 - cellmage.storage.memory_store - INFO - Saved conversation to memory with ID: gemini_recursion_conversation
2025-04-24 07:08:05,416 - cellmage.history_manager - INFO - History cleared. Kept 2 system messages.
2025-04-24 07:08:05,421 - cellmage.history_manager - INFO - Loaded conversation from gemini_recursion_conversation with 10 messages


Conversation saved to: gemini_recursion_conversation
History cleared, current message count: 2
Conversation loaded, message count: 10

===== Loaded Conversation Summary =====
[user] What is a recursive function in programming?
[assistant] Okay, let's break down what a recursive function i...
[user] Can you give me an example using Python?
[assistant] Okay, let's use the **Fibonacci sequence** as a cl...
[user] What are some ways to optimize it to avoid stack o...
[assistant] Okay, let's look at ways to optimize the recursive...
[user] Implement one of those optimization techniques in ...
[assistant] Okay, let's implement the **Memoization** techniqu...


## Conclusion

This notebook has demonstrated the capabilities of Cellmage with the Gemini 2.5 Flash model, including:

1. Setting up a Gemini model configuration
2. Creating specialized personas
3. Testing code generation capabilities
4. Testing creative content generation
5. Using code snippets to provide context
6. Testing contextual understanding across multiple conversation turns
7. Saving and loading multi-turn conversations

The Gemini 2.5 Flash model shows strong capabilities across both technical and creative tasks within the Cellmage framework.